# Test algorithms on a synthetic realization of a multivariate Wold process

In this notebook, we run all the algorithms described on the paper on a synthetic realization of a multivariate Wold process. Specifically, the algorithms are:

* `WoldModelVariational`: The VI approach introduced in the paper (denoted VI in the paper).
* `WoldModelBBVI`: Black-box VI (denoted BBVI in the paper).
* `GrangerBusca`: The Granger-Busca approach (denoted GB in the paper).
* `WoldModelMLE`: Maximum-Likelihood estimation (denoted MLE in the paper).

---

Import the libraries of interest for this notebook

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

import torch
import numpy as np

# Import models
import lib
from lib.models import WoldModelVariational, WoldModelBBVI, WoldModelMLE
from gb import GrangerBusca

# Set numpy print format
np.set_printoptions(precision=2, floatmode='fixed', sign=' ')

# Set larger cells for nicer output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

Set random seed for all experiments

In [ ]:
seed = np.random.randint(2**32-1)
gen_seed = 1234567
sim_seed = 8765



---

## Generate toy example dataset


In [ ]:
# Number of dimensions
dim = 10
# Number of events to simulate
max_jumps = 10e3 * dim

# Generate a set of parameters for the process
param_dict = {
    'baseline': np.random.uniform(0.0, 0.05, size=dim),
    'adjacency': (np.random.uniform(0.1, 0.2, size=(dim, dim)) 
                  * np.random.binomial(n=1, p=2*np.log(dim)/dim, size=(dim, dim))),
    'beta': np.random.uniform(0.0, 1.0, size=(dim, dim))
}

print('Baseline:')
print(param_dict['baseline'].round(2))
print('Alpha:')
print(param_dict['adjacency'].round(2))
print('Beta:')
print(param_dict['beta'].round(2))

### Simulate a realization

In [ ]:
print('  - Simulate a realization...')
wold_sim = lib.simulate.MultivariateWoldSimulator(**param_dict)
events = wold_sim.simulate(max_jumps=max_jumps, seed=sim_seed)
end_time = max(map(max, events))
print((f"    - Simulated {sum(map(len, events)):,d} events "
       f"with end time: {end_time:.2f}"))
print("    - Events:")
for i, events_i in enumerate(events):
    print(f"      - dim {i:>2d} ({len(events_i):>5d} events):", events_i)

---

## Run inference algorithms

## Run VI

Define the parameters of the prior.

In [ ]:
dim = len(events)
# Set model
vi_model = WoldModelVariational(verbose=True)
vi_model.observe(events)
# Set priors
# prior: Alpha
as_pr = 0.1 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))
# prior: Beta
bs_pr = 10.0 * np.ones((dim, dim))
br_pr = 10.0 * np.ones((dim, dim))
# prior: Z
zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]
# Set callback (to monitor evolution of iterations)
callback = lib.utils.callbacks.Callback(x0=(as_pr / ar_pr)[1:, :].flatten(), print_every=10, dim=dim)
# Fit model
conv = vi_model.fit(as_pr=as_pr, ar_pr=ar_pr, bs_pr=bs_pr, br_pr=br_pr,
                    zc_pr=zc_pr, max_iter=20000, tol=1e-4,
                    callback=callback)

In [ ]:
print('Model:', type(vi_model).__name__)

# Mean of posterior
vi_coeffs_mean = vi_model.alpha_posterior_mean()
vi_base_mean = vi_coeffs_mean[0,:]
vi_adj_mean = vi_coeffs_mean[1:,:]

print('\nBaseline:')
print('---------')
print('Ground truth:')
print(param_dict['baseline'])
print('Estimated:')
print(vi_base_mean)

print('\nAdjacency:')
print('---------')
print('Ground truth:')
print(param_dict['adjacency'])
print('Estimated:')
print(vi_adj_mean)

print('\nMetrics:')
print('---------')
fsc = lib.utils.metrics.fscore(vi_adj_mean.flatten(), param_dict['adjacency'].flatten(), threshold=0.05)
print(f'F1-Score: {fsc:.2f}')
relerr = lib.utils.metrics.relerr(vi_adj_mean.flatten(), param_dict['adjacency'].flatten())
print(f'Relative Error: {relerr:.2f}')
precAt10 = lib.utils.metrics.precision_at_n(vi_adj_mean.flatten(), param_dict['adjacency'].flatten(), n=10)
print(f'Precision@10: {precAt10:.2f}')

## Run GB

In [ ]:
# Set random seed
np.random.seed(seed)

# Define model
granger_model = GrangerBusca(
    alpha_prior=1.0/len(events),
    num_iter=3000,
    metropolis=True,
    beta_strategy='busca',
    num_jobs=1,
)
granger_model.fit(events)

In [ ]:
print('Model:', type(granger_model).__name__)

# Extract infered adjacency from the model
gb_adj_hat = granger_model.Alpha_.toarray()
gb_adj_hat = gb_adj_hat / gb_adj_hat.sum(axis=1)

print('\nBaseline:')
print('---------')
print('Ground truth:')
print(param_dict['baseline'])
print('Estimated:')
print(granger_model.mu_)

print('\nAdjacency:')
print('---------')
print('Ground truth:')
print(param_dict['adjacency'])
print('Estimated:')
print(gb_adj_hat)

print('\nMetrics:')
print('---------')
fsc = lib.utils.metrics.fscore(gb_adj_hat.flatten(), param_dict['adjacency'].flatten(), threshold=0.05)
print(f'F1-Score: {fsc:.2f}')
relerr = lib.utils.metrics.relerr(gb_adj_hat.flatten(), param_dict['adjacency'].flatten())
print(f'Relative Error: {relerr:.2f}')
precAt10 = lib.utils.metrics.precision_at_n(gb_adj_hat.flatten(), param_dict['adjacency'].flatten(), n=10)
print(f'Precision@10: {precAt10:.2f}')


## Run BBVI

In [ ]:
# set random seed
np.random.seed(seed)
torch.manual_seed(seed)

# Set initial point
coeffs_start = torch.tensor(np.hstack((
    # loc
    -2.0 * torch.ones(dim, dtype=torch.float),                  # baseline
    0.0 * torch.ones((dim, dim), dtype=torch.float).flatten(),  # beta
    0.0 * torch.ones((dim, dim), dtype=torch.float).flatten(),  # adjacency
    # scale
    torch.log(0.2 * torch.ones(dim, dtype=torch.float)),
    torch.log(0.2 * torch.ones((dim, dim), dtype=torch.float).flatten()),
    torch.log(0.2 * torch.ones((dim, dim), dtype=torch.float).flatten()),
)))

# Define priors/posteriors
posterior = lib.posteriors.LogNormalPosterior
prior = lib.priors.GaussianLaplacianPrior
mask_gaus = torch.zeros(dim + dim**2 + dim**2, dtype=torch.bool)
mask_gaus[:dim + dim**2] = 1  # Gaussian prior for baseline and beta
C = 1e3

# Init the model object
bbvi_model = WoldModelBBVI(posterior=posterior, prior=prior, C=C,
                      prior_kwargs={'mask_gaus': mask_gaus},
                      n_samples=1, n_weights=1, weight_temp=1,
                      verbose=False, device='cpu')
bbvi_model.observe(events, end_time)

# Set the callback
callback = lib.utils.callbacks.Callback(x0=coeffs_start, print_every=10, dim=dim)

# Fit the model
conv = bbvi_model.fit(x0=coeffs_start, optimizer=torch.optim.Adam, lr=0.1,
                      lr_sched=0.9999, tol=1e-4, max_iter=20000,
                      mstep_interval=100, mstep_offset=500, mstep_momentum=0.5,
                      seed=None, callback=callback)

In [ ]:
print('Model:', type(bbvi_model).__name__)

# Extract infered adjacency from the model
bbvi_coeffs_mean = bbvi_model.posterior.mean(bbvi_model.coeffs[:bbvi_model.n_params],
                                             bbvi_model.coeffs[bbvi_model.n_params:]).detach().numpy()

bbvi_base_mean = bbvi_coeffs_mean[:dim]
bbvi_adj_mean = bbvi_coeffs_mean[-dim**2:]

print('Baseline:')
print('---------')
print('Ground truth:')
print(param_dict['baseline'])
print('Estimated:')
print(bbvi_base_mean)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(param_dict['adjacency'])
print('Estimated:')
print(bbvi_adj_mean)

print()
print('Metrics:')
print('---------')
fsc = lib.utils.metrics.fscore(bbvi_adj_mean.flatten(), param_dict['adjacency'].flatten(), threshold=0.05)
print(f'F1-Score: {fsc:.2f}')
relerr = lib.utils.metrics.relerr(bbvi_adj_mean.flatten(), param_dict['adjacency'].flatten())
print(f'Relative Error: {relerr:.2f}')
precAt10 = lib.utils.metrics.precision_at_n(bbvi_adj_mean.flatten(), param_dict['adjacency'].flatten(), n=10)
print(f'Precision@10: {precAt10:.2f}')


## Run MLE

In [ ]:
# set random seed
np.random.seed(seed)
torch.manual_seed(seed)

# Set initial point
coeffs_start = torch.tensor(np.hstack((
    np.random.uniform(0.0, 1.0, size=dim),     # baseline
    np.random.uniform(0.0, 1.0, size=dim**2),  # beta
    np.random.uniform(0.0, 1.0, size=dim**2)   # adjacency
)))

# Define model
mle_model = WoldModelMLE(verbose=True)
mle_model.observe(events, end_time)

# Set callback (to monitor algorithm)
callback = lib.utils.callbacks.Callback(coeffs_start, print_every=10, dim=dim)

# Fit model
conv = mle_model.fit(x0=coeffs_start, optimizer=torch.optim.Adam, lr=0.1,
                     lr_sched=0.9999, tol=1e-4, max_iter=20000,
                     penalty=lib.priors.GaussianPrior, C=1e10,
                     seed=None, callback=callback)
mle_coeffs_hat = mle_model.coeffs.detach().numpy()

In [ ]:
print('Model:', type(mle_model).__name__)

# Extract infered adjacency from the model
mle_coeffs_hat = mle_model.coeffs.detach().numpy()
mle_base_hat = mle_coeffs_hat[:dim]
mle_alpha_hat = np.reshape(mle_coeffs_hat[dim+dim**2:], (dim, dim))

print('Baseline:')
print('---------')
print('Ground truth:')
print(param_dict['baseline'])
print('Estimated:')
print(mle_base_hat)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(param_dict['adjacency'])
print('Estimated:')
print(mle_alpha_hat)

print()
print('Metrics:')
print('---------')
fsc = lib.utils.metrics.fscore(mle_alpha_hat.flatten(), param_dict['adjacency'].flatten(), threshold=0.05)
print(f'F1-Score: {fsc:.2f}')
relerr = lib.utils.metrics.relerr(mle_alpha_hat.flatten(), param_dict['adjacency'].flatten())
print(f'Relative Error: {relerr:.2f}')
precAt10 = lib.utils.metrics.precision_at_n(mle_alpha_hat.flatten(), param_dict['adjacency'].flatten(), n=10)
print(f'Precision@10: {precAt10:.2f}')